# Analyzing Algorithmic Complexity

This notebook serves to find the complexity of each algorithm explored in the general case.

## Brute-Force

The brute-force approach is the most basic approach to solving the TSP, but also one of the slowest. It tries each path and finds which one is the fastest.

Our solution, as implemented in `brute_force.py`, runs in $O(n!)$ time. We will first prove that `_brute_force_rec` runs in $O(n!)$ time, then show that `brute_force` does nothing to increase the time complexity.

Let $P(n_p)$ be the proposition that `_brute_force_rec` runs in $O(n_p!)$ time, where $n_p$ is the number of non-0 nodes in `graph` (i.e. the number of nodes to consider) minus the length of `path`. We can see that this is true for $n_p=1$, as it runs in constant time (i.e. no loops are encountered). This is the case as, if $n_p=1$, the function will immediately enter the first `if` statement and then return, running in constant time. Constant time is $O(1)$, which is equal to $O(n_p!)$ when $n_p=1$.

We will now show that if $P(k)$ is true for some $k \ge 1$, then $P(k+1)$ is also true. Since $k+1 \ge 2$, we know that the function will bypass the first `if` statement and enter the loop. This loops repeats `len(graph) - 1` times. However, there are two options - `len(path)` of the loops run in constant time (as the only thing it does is check if `node` is in `path` - a constant-time operation since `path` is a set - and then move onto the next loop), while the remaining `k+1` loops are more complicated. Here's what it does line-by-line:

1. Check if `node` is in `path`. As `path` is a set, this takes constant time.
2. Add `node` to `path`. This also takes constant time.
3. Recurse. In this new call, as `path` is now one element bigger, $n_p=k$. By our inductive hypothesis, we know that this runs in $O(k!)$ time.
4. Remove `node` from `path`. This takes constant time.
5. Create `act_dist` as the sum of `dist` and the distance from `last_node` to `node`. This lookup and addition takes constant time.
6. Check if this produced a shorter path. Each comparison (along with updating `shortest_dist` and `shortest_path`) takes constant time.

Since we only care about the overall behavior in terms of $k+1$, we can ignore any constant-time operations, as they are negligible. This means each loop where `node` is not in `path` runs in $O(k!)$ time.

There are `len(path)` loops that run in constant time and $k+1$ loops that run in $k!$ time. This means, in total, the runtime of `brute_force` for $k+1\ge2$ is `len(path)`$+(k+1)k!$. As we only care about the long-term behavior, `len(path)` is effectively zero, leaving the runtime to be $0+(k+1)k!$. This simplifies to $O((k+1)!)$. Therefore, we have shown that if $P(k)$ is true for some $k \ge 1$, then $P(k+1)$ is also true. Since we have also shown $P(1)$, by the principle of mathematical induction, $P(n_p)$ is true for all $n_p \ge 1$. That is, `_brute_force_rec` runs in $n_p!$ time where $n_p$ is the number of non-zero nodes in `graph` minus the number of nodes in `path`.

We will now show that `brute_force` has this same time complexity, however this time in terms of $n$, the number of non-zero nodes in `graph`. `brute_force` only has two lines. We can see that the first line, which calls `_brute_force_rec` runs in $O(n!)$ time, as it passes in an empty set. That is $n_p=$`len(graph)`$-1-0=n$. The second statement returns the distance plus a constructed list. The constructed list involves a list reversal and an effective `extend` call, both of which run in linear time. The runtime of `brute_force` is therefore $O(n!+2n)$. Since we only care about the long-term behavior, we can ignore the $2n$ as it is negligible. Therefore, the true runtime of `brute_force` is $O(n!)$